In [1]:
%load_ext rich

# Pre-built modules

In [2]:
import promptimus as pm

In [3]:
provider = pm.llms.OllamaProvider(model_name="phi4", base_url="http://lilan:11434/v1")

## Memory Module

In [4]:
assistant = pm.modules.MemoryModule(
    memory_size=3, system_prompt="You are an assistant."
).with_provider(provider)

In [5]:
await assistant.forward("Hi my name is Ailadin!")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Hello, Ailatin! It's nice to meet you. How can I assist you today?"
)

In [6]:
# remembering name from context
await assistant.forward("What is my name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Your name is Ailadin. Is there anything else you would like to know or discuss?'
)

In [7]:
await assistant.forward("What is your name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="I'm Phi, a language model developed by Microsoft. How can I assist you today? If you have any questions or need information, feel free to ask!"
)

In [8]:
await assistant.forward("What is my the weather today?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="I'm unable to provide real-time data like current weather conditions as I don't have access to live databases. However, I recommend checking a reliable weather forecasting service such as The Weather Channel, BBC Weather, or using an app like Weather.com or AccuWeather on your smartphone. You can also ask Alexa, Google Assistant, or Siri for quick updates on the weather in your area. If you need guidance on how to find this information, feel free to ask!"
)

In [9]:
# memory maxlen exhausted
assistant.memory

Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="I'm Phi, a language model developed by Microsoft. How can I assist you today? If you have any questions or need information, feel free to ask!"), Message(role=<MessageRole.USER: 'user'>, content='What is my the weather today?'), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="I'm unable to provide real-time data like current weather conditions as I don't have access to live databases. However, I recommend checking a reliable weather forecasting service such as The Weather Channel, BBC Weather, or using an app like Weather.com or AccuWeather on your smartphone. You can also ask Alexa, Google Assistant, or Siri for quick updates on the weather in your area. If you need guidance on how to find this information, feel free to ask!")], maxlen=3)]

In [10]:
# now it doesn't have memory in context
await assistant.forward("Can you recall my name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="I'm sorry, but I don't have access to personal data about individuals unless it has been shared with me during our conversation for the purpose of assisting. My design prioritizes user privacy and security, ensuring that no personal information is stored or remembered after our interactions. If you’d like assistance on other topics, feel free to ask!"
)

In [11]:
# memory can be cleaned manually
assistant.memory.reset()
assistant.memory

Memory[deque([], maxlen=3)]

In [12]:
# or can be cleared on context enter and exit

with assistant.memory:
    await assistant.forward("Hi my name is Ailadin")
    await assistant.forward("What is my name")
    print("Inside context", assistant.memory)
print("----")
print("Outside context", assistant.memory)

Inside context Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="Hello Ailadin! How can I assist you today? Let me know if there's anything specific you'd like to ask or discuss. 😊\n\n"), Message(role=<MessageRole.USER: 'user'>, content='What is my name'), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Your name is Ailadin, as mentioned in your introduction earlier. If you have any questions or need assistance, feel free to let me know! How can I help you today? 🌟 \n\n')], maxlen=3)]
----
Outside context Memory[deque([], maxlen=3)]


In [13]:
assistant.save("assets/step_3_memory.toml")
!cat assets/step_3_memory.toml

prompt = """
You are an assistant.
"""



## Structural Output

In [14]:
from enum import StrEnum, auto

from pydantic import BaseModel, Field

In [15]:
class Operation(StrEnum):
    SUM = auto()
    SUB = auto()
    DIV = auto()
    MUL = auto()


class CalculatorSchema(BaseModel):
    reasoning: str
    a: float = Field(description="The left operand.")
    b: float = Field(description="The right operand.")
    op: Operation = Field(description="The operation to execute.")


module = pm.modules.StructuralOutput(CalculatorSchema).with_provider(provider)

In [16]:
await module.forward("I have 10 cows, I need twice the amount")


CalculatorSchema(
    reasoning='To find twice the amount of 10 cows, we perform multiplication with 2.',
    a=10.0,
    b=2.0,
    op=<Operation.MUL: 'mul'>
)

In [17]:
await module.forward("I have 10 cows, i need half of them")


CalculatorSchema(
    reasoning="The question involves dividing the number of cows by two to find out how many constitute 'half'.",
    a=10.0,
    b=2.0,
    op=<Operation.DIV: 'div'>
)

In [18]:
await module.forward("I have 11.11 cows. and 11 left the herd")


CalculatorSchema(
    reasoning='The total number of cows after 11 leave from an initial count of 11.11 is calculated by subtraction: 11.11 - 11.',
    a=11.11,
    b=11.0,
    op=<Operation.SUB: 'sub'>
)

In [19]:
await module.forward("I have 5 cows. and my friend have 7")


CalculatorSchema(
    reasoning='The number of cows owned by the user is added to the number of cows owned by their friend.',
    a=5.0,
    b=7.0,
    op=<Operation.SUM: 'sum'>
)

In [20]:
module.save("assets/step_3_structural.toml")
!cat assets/step_3_structural.toml

retry_template = """
Your response does not conform to the required schema. Please correct your output by ensuring it matches the expected format and constraints. 

**Schema Validation Error:**  
`{error_message}`  
  
Please reattempt the response, ensuring strict adherence to the schema.    
"""


[predictor]
prompt = """
You are designed to generate structured JSON outputs adhering to a predefined schema without any embellishments or formatting.

Schema Description: {'$defs': {'Operation': {'enum': ['sum', 'sub', 'div', 'mul'], 'title': 'Operation', 'type': 'string'}}, 'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'a': {'description': 'The left operand.', 'title': 'A', 'type': 'number'}, 'b': {'description': 'The right operand.', 'title': 'B', 'type': 'number'}, 'op': {'$ref': '#/$defs/Operation', 'description': 'The operation to execute.'}}, 'required': ['reasoning', 'a', 'b', 'op'], 'title': 'CalculatorSchema', 'type': 'object'}

Response Guidelines:
- Ensu